This notebook was provided by Dr. Ryan Urbanowicz to be used during the Math, Engineering and Technology at CHOP (METACHOP) Workshop on June 19th, 2019.

*** 
# METACHOP Workshop
* Contributors: Jorge Guerra, Helen Loeb and Remo Williams
*  Affiliation: Children's Hospital of Philadelphia
* Date: 05/20/2019
* github: https://github.com/guerrajorge/metachop

***
# Machine Learning (ML) 102 Workshop
* Author: Ryan Urbanowicz, PhD (Debugging assistance by Dr. Trang Le)
*  Affiliation: University of Pennsylvania - Department of Biostatistics, Epidemiology, and Informatics & Institute for Biomedical Informatics (IBI) in collaboration with the Leonard Davis Institute (LDI)
* Date: 5/1/19
* github: https://github.com/UrbsLab/ML_Pipeline_Notebooks

A recording of Machine learning 101 Workshop (Dec 2018) – An Introduction to Machine Learning is available at:  
https://bluejeans.com/playback/s/PF3d7xdm3DSBbZgHw6JpHnhoVSPVg2ACytbA6eMKFHRWEXSV2UaFNHMXJn7GV9kN

A recording of Machine Learning 102 Workshop (May 2019) – Machine Learning: An Analysis Pipeline is available at:
https://bluejeans.com/playback/s/sYL8Nfeq9M1H42nLcGPxuxc59aj1DZI6o3Qf8EYnApXP1W2vnphICfuuxlsokPIF

To easily view this Jupyter Notebook in Google collab, use the following link: 
https://colab.research.google.com/github/UrbsLab/ML_Pipeline_Notebooks/blob/master/ML_102_Workshop.ipynb

***


<img src="images/python_jupyter.png" />

***
## Introduction
This notebook presents an example of a machine learning analysis pipeline from start to finish. It was written to be paired with the ML 102 Workshop presented in collaboration with IBI and LDI, and it was modified for the CHOP METACHOP workshop. Please note that this notebook is meant to present an accessible example, but does not necessarily include the optimal strategies to analyze the target dataset examined herein. Identifying the optimal analysis pipeling steps/components is one of the fundamental challenges of data science.  This is almost never known ahead of time when seeking to tackle a new dataset/anlaysis. The pipeline presented below could be reproduced using different software or coding languages.  We have opted to utilize Python and the Jupyter notebook framework here due to is accessibility, flexibility, and prevalence in the ML community. 

For the purposes of this workshop this notebook is primarily meant to be viewed as an html link (pre-run) as a reference/resource/example.  This avoids the many possible complications related to installing python, the necessary packages, understanding how to use/run jupyter notebooks, etc.  However, both the html and original jupyter notebook file are available for those who wish to try out the notebooks themselves.  However explaining how to code in python or the logistics of how to run this notebook interactively are beyond the intended scope of the workshop itself ??

<img src="images/ds_pipeline.png" />

***
The following warning will be displayed when trying to run the notebook:

<img src="images/authorization.png" />

Unselect __"Reset all runtimes before running"__ and Click on __"RUN ANYWAY"__

#### Import Necessary Python Packages

In [ ]:
# Need to install the following packages (not naitive to google colab)
!pip install skrebate

#Basic Packages
import os
import random
import pandas as pd
import numpy as np
import scipy.stats as scs
from scipy.stats import randint
import re

#Scikit-Learn Packages:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif # Mutual information for a discrete target.
from sklearn import tree #import decision tree package
from sklearn import metrics #import evaluation metric package
from sklearn.ensemble import RandomForestClassifier #import decision tree package
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from sklearn.utils.multiclass import unique_labels

# In-depth data exploration package
import pandas_profiling


#ReliefF feature selection package
from skrebate import ReliefF

#Visualization Packages:

#%matplotlib notebook
%matplotlib inline   
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import _tree
from matplotlib import pyplot
import graphviz 


# # Jupyter Notebook Hack: This code ensures that the results of multiple commands within a given cell are all displayed, rather than just the last. 
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

#Set a random seed for the notebook so that individual runs of the notebook yield the same results
randSeed = 99 #changing this value will potentially change the models and results due to stochastic elements of the pipeline. 
np.random.seed(randSeed)

#After running this cell, any error message here will inform you what package still needs to be installed on your system using pip install or conda install.

***
# RAW DATA
Here we describe our target dataset, load it, and examine some basic properties of the data.  This examination of the data can be considered part of the exploratory analysis.  We have included it in this first section to provide a more logical flow to this analysis pipeline. 

***
### Description of Raw Target Dataset


For the purpose of this notebook we have selected an accessible open source dataset from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). Specifically we will apply our ML pipeline to classification dataset gathered to try and predict one year survival in patients with hepatocellular carcinoma (HCC). [Target Dataset Source](http://archive.ics.uci.edu/ml/datasets/HCC+Survival).

<img src="images/hcc_dataset_description.png" />

* OpenSource: https://archive.ics.uci.edu/ml/datasets/HCC+Survival

Prior to loading the data here we opened the dataset in excel as well as in a text editor noting the following about the dataset: 
1. The data has comma separated values (i.e. csv format).
2. There is no header (i.e. column labels) in the data.
3. A secondary data dictionary file is available that describes the features and includes the header values
4. Missing values are denoted with '?'
5. From the dictionary file we know that the class/outcome column is named 'Class Attribute'.
6. Oddly the minority class is coded as 0 (patient died), and the majority class is coded as 1 (patient alive).  This is because the 'target' event in this data is 'patient survived 1 year'. 

We have created a csv file of the header names in excel taken directly from this data dictionary.  GitHub links to the data files needed for this notebook are included below:

* [Header File](https://raw.githubusercontent.com/guerrajorge/METACHOP/master/data/HCC_headers.txt)
* [Data File](https://raw.githubusercontent.com/guerrajorge/METACHOP/master/data/hcc-data.txt)

***
### Load Data

In [ ]:
# load header names file
# use the 'raw location' url to access the files from github 
header_file = 'https://raw.githubusercontent.com/guerrajorge/METACHOP/master/data/HCC_headers.txt'
headers = pd.read_csv(header_file, sep='\t',header=None) 
print('Number of (rows, columns)')
print(headers.shape)
print('First 5 columns:')
print(headers.values[0,:5].tolist())
header_list = headers.values[0].tolist() #creates a list variable from the first element dataframe

In [ ]:
# load dataset and provide header names from above.
target_data_file = 'https://raw.githubusercontent.com/guerrajorge/METACHOP/master/data/hcc-data.txt'
hcc_df = pd.read_csv(target_data_file, na_values='?', header=None, names = header_list) # Data loaded so that blank excell cells are 'NA'
print('Number of (rows, columns)')
print(hcc_df.shape)

In [ ]:
num_instances = hcc_df.shape[0] # number of examples
num_features = hcc_df.shape[1] # number of features

print('Dataset contains {0} instances.'.format(num_instances))
print('Dataset contains {0} features plus 1 class/outcome.'.format(num_features-1))

***
### Examine Basic Data Properties
Run some basic Pandas commands to examine/confirm dataset properties. 

In [ ]:
#Examine the first 5 rows of the loaded data
hcc_df.head()

Remember class/outcome column is named '__Class Attribute__' coded as 0 (patient died), and  1 (patient alive)

In [ ]:
# method prints information about a DataFrame including the index dtype and column dtypes, non-null values and memory usage
hcc_df.info()

***
### Assess Missingness in Data

In [ ]:
#Evaluate missingness and data availability
print("Missing Value Counts")
missing_count = hcc_df.isnull().sum()
missing_count

* We confirm here that there is no missing data in the class/outcome variable. If there had been we would have to remove any rows with missing outcome later in the cleaning section. This is because we are performing supervised learning, i.e.  (label/outcome) required for modeling. 

In [ ]:
# examine the number of unique values for each variable/feature. Note that missing values are not being include as unique values. 
unique_count = hcc_df.nunique()
unique_count

In [ ]:
#Plot a histogram of these unique variable counts. 
ax = unique_count.hist(bins=num_instances,figsize=(16,4))
ax.set_xlabel("Unique Value Counts")
ax.set_ylabel("Frequency")
ax.set_title("Histogram of Unique Value Counts In Feature Set")

* We observe that nearly half the features are binary, and there are some features that appear to be discrete interger values, and a number of real-valued features. 

In [ ]:
#Plot a histogram of the missingness observed over all features in the dataset
ax = missing_count.hist(bins=num_instances,figsize=(16,4))
ax.set_xlabel("Missing Value Counts")
ax.set_ylabel("Frequency")
ax.set_title("Histogram of Missing Value Counts In Feature Set")

Side by side comparison

<img src="images/hcc_unique_missing.png" />

***
# Yet, another package to examine the data

Package: pandas_profiling  
https://github.com/pandas-profiling/pandas-profiling

In [ ]:
profile = pandas_profiling.ProfileReport(hcc_df)
profile.to_file('exploratory_plots/hcc_profile_result.html')

In [ ]:
# For large datasets the analysis can run out of memory, or hit recursion depth constraints; 
# especially when doing correlation analysis on large free text fields
pandas_profiling.ProfileReport(hcc_df)

***
# PREPROCESSING
Every unique dataset and analysis comes with it's own characteristics and challenges.  Therefore preprocessing requires an understanding of the target data and analysis goal and may require fewer, additional, or alternative steps than what we describe here. Also note that an exploratory analysis and data cleaning go hand in hand and are often completed together. 

***
## Data Cleaning

In [ ]:
print('initial header list')
print(hcc_df.columns)

# conver the headers to lower case
hcc_df.columns = [x.lower() for x in hcc_df.columns]

# replace spaces
hcc_df.columns = [x.replace(" ", "_") for x in hcc_df.columns]

# remove units
hcc_df.columns = [re.sub(r"_*\(.*\)","", x) for x in hcc_df.columns]

# remove special characters if located at the end of the string i.e. * in international_normalised_ratio* or _ in symptoms_
hcc_df.columns = [re.sub(r"[^a-zA-Z0-9]$","",x) for x in hcc_df.columns]

print('')
print('resulting header list')
print(hcc_df.columns)

### Rename outcome variable

In [ ]:
hcc_df['target'] = hcc_df['class_attribute']
hcc_df.drop('class_attribute', axis=1, inplace=True)

### Remove Rows
* remove any rows that have a missing outcome variable value. 

### Remove Columns
* Remove unneeded or clearly irrelevant columns such as instance id. 
* Prevent [data leakage](https://machinelearningmastery.com/data-leakage-machine-learning/)
    * Remove precursor features (used to build outcome variable in study)
    * Remove features that would be unavailable when prediction made.
We have reviewed the data dictionary for our target dataset and have found no columns that need to be removed for this analysis. This will be skipped. 

### Deal with Missing Data

<img src="images/missingness.png" />


In [ ]:
for cols in hcc_df.columns:
    # Median-Value Imputation (For continuous-valued features)
    if hcc_df[cols].nunique() > 10: #10 chosen as a convenient cutoff for discriminating discrete from continuous variables. 
        hcc_df[cols].fillna(hcc_df[cols].median(), inplace=True)
    # Mode-Value Imputation (For discrete-valued features)
    else:
        hcc_df[cols].fillna(hcc_df[cols].mode().iloc[0], inplace=True)

In [ ]:
#Examine the number of unique values for each variable/feature. 
unique_count = hcc_df.nunique()
unique_count

#Re-evaluate missingness and data availability
print("Missing Value Counts")
missing_count = hcc_df.isnull().sum()
missing_count

### Cast Variable Types (as needed)
We examine how the variable types are being automatically detected by the Pandas package to either confirm that they are correct or to 'cast' variables so that Pandas and any other applied algorithms will know how to handle each variable when necessary. 

In [ ]:
hcc_df.dtypes

* Based on the data dictionary a number of variables have been assigned the wrong data type by Pandas.  We will remedy this below. 

* Before moving forward we will first recode our class variable follow the more conventional data standard of the 'positive' class being the minority class.  This will help with downstream evaluation interpretation. We are effectively rephrasing the predictive goal for this data to 'predicting the target event of a patient dying' which represents the miniority class. From here on out, class 0 = survived 1 year while class 1 = died. 


In [ ]:
# previously Class Attribute
outcome_name = 'target'

In [ ]:
#Recode class values (0's to 1's and 1's to 0's) 
hcc_df[outcome_name]=hcc_df[outcome_name].replace(to_replace=0, value=2)
hcc_df[outcome_name]=hcc_df[outcome_name].replace(to_replace=1, value=0)
hcc_df[outcome_name]=hcc_df[outcome_name].replace(to_replace=2, value=1) 

# # similar logic
# hcc_df['target'] = ~ hcc_df['target'].astype(bool)

#Grab column names as a list
header = list(hcc_df)

#Cast variable types as needed. It is useful here to specifiy categorical variables here as 'object' for the exploratory analysis.
hcc_df[header[0:23]] = hcc_df[header[0:23]].astype(dtype='object')
hcc_df[['age_at_diagnosis']] = hcc_df[['age_at_diagnosis']].astype(dtype='float64')
hcc_df[['performance_status']] = hcc_df[['performance_status']].astype(dtype='float64')
hcc_df[[outcome_name]] = hcc_df[[outcome_name]].astype(dtype='object')

#Confirm correct casting of variable types. 
hcc_df.dtypes

### Class Imbalance
Determine what magnitude (if any) of [class imbalance](http://www.chioka.in/class-imbalance-problem/) exists in this dataset. Classes are considered to be 'balanced' if there are an equal number of instances within each class.  Class imbalance can be accounted for by applying the proper evaluation metrics downstream.  Generally speaking machine learning methods are most successful when training on more 'balanced' datasets.  Datasets can be artificially proprocessed to be more balanced using [oversampling and undersampling](https://en.wikipedia.org/wiki/Oversampling_and_undersampling_in_data_analysis) methods.  

In [ ]:
print("Counts of each class")
hcc_df[outcome_name].value_counts()
hcc_df[outcome_name].value_counts().plot(kind='bar')
plt.ylabel('Count')
plt.title('Class Counts (Checking for Imbalance)')

* We observe some class imbalance (1:1.61) where there are more 1's than 0's, where 1=lived, and 0=died.
* Goal = 'predicting the target event of a patient dying'
* class 0 = survived 1 year, class 1 = died. 

### Outliers
Our knowledge of the dataset and it's target domain should be applied to perform a manual quality control check of the data.  Specifically do we observe any [outliers](https://en.wikipedia.org/wiki/Outlier) in any of the variables of the dataset? For instance we might check the 'Age at Diagnosis' variable and confirm that we don't see any ages outside of what would be reasonable for this target study (e.g. it's highly unlikely to observe anyone over the age of 110). Obvious highly unlikely or impossible outliers should be removed (i.e. either treated as a missing value, or the entire instance removed). These are often typos during data entry. 

* We don't observe any obvious 'impossible' outliers that need to be removed, however some statistical outliers are observed in the boxplots and in the basic descriptive summary statistics above.  For the purposes of this analysis we will not remove any of these outliers, however this is a consideration for followup analysis. 

***
## Data Partitioning
In order to rigourously evaluate our downstream predictive modeling it is important to partition our entire datasets (at minimum) into a training as well as a testing dataset. Even better if you have a sufficient number of instances in your dataset, it may be split into [training, validation, and testing sets](https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7). Testing data will not be applied until modeling is complete in order to evaluate our model on data it has not yet seen.  This is critical to properly evaluating predictive success. 

<img src="images/training_validation_testing.png" />

A more rigorous approach is to perform [k-fold cross validation](https://machinelearningmastery.com/k-fold-cross-validation/), where analyses are performed on resampled partitions of the data, such that all instances serve as part of the testing set in at least one of the partitions.  This stragegy helps to avoid the potential bias introduced by evaluating a model on one random training/testing partition. 

<img src="images/kfold.png" />


For the purposes of notebook simplicity we will employ a single basic training/testing partition of the dataset. 

* There are many ways to go about partitioning and different implementations of this in Python. Some built in cross validation methods exist in scikit-learn but they do not easily allow you to save your respective training and testing datasets as files for external replication. 
* Often it may be easier to perform certain feature processing steps (e.g. feature transformation, feature engineering, and feature construction) prior to completing data partitioning. For the purposes of this notebook it is convenient to introduce and perform data partitioning here under 'Preprocessing'. 

<img src="images/train_test.png" />


In [ ]:
#For the purposes of downstream ML recast all features previously assigned to be 'object' to 'int'
hcc_df[header[0:23]] = hcc_df[header[0:23]].astype(dtype='int')
hcc_df[['target']] = hcc_df[['target']].astype(dtype='int')
# hcc_df.dtypes
#Partition data into a training and testing set using convenient scikit-learn command.
train, test = train_test_split(hcc_df, test_size=0.2, stratify=hcc_df[outcome_name],random_state=randSeed) # 20% test 
# set size, stratify option used to ensure class ratio is maintained in the partitions, random seed specified for reproducibility
print('Confirm dimensions of training set')
print(train.shape)
print('Confirm dimensions of testing set')
print(test.shape) #Confirm dimensions of testing set

* We confirm below that both the training and test sets have preserved the original class balance to help avoid this additional bias. 

In [ ]:
print("Counts of each class in training data")
train[outcome_name].value_counts()
train[outcome_name].value_counts().plot(kind='bar')
plt.ylabel('Count')
plt.title('Class Counts (Checking for Imbalance)')

In [ ]:
print("Counts of each class in testing data")
test[outcome_name].value_counts()
test[outcome_name].value_counts().plot(kind='bar')
plt.ylabel('Count')
plt.title('Class Counts (Checking for Imbalance)')

In [ ]:
#Split dataframe into features and outcome (standard format for scikit-learn methods)
x_train = train.drop(outcome_name, axis=1).values
y_train = train[outcome_name].values

#Later when evaluating models we will need y_test so we will create it now...
y_test = test[outcome_name].values
x_test = test.drop(outcome_name, axis=1).values

# MODELING
This is the stage that everyone thinks of when it comes to machine learning and data mining. 

## Method Selection
As a general rule of thumb it is typically best practice to run a number of machine learning algorithms in modeling (i.e. at least 2-3), and ideally those methods should have differen strengths and weaknesses.  This is due to the [no-free-lunch theorem](https://en.wikipedia.org/wiki/No_free_lunch_theorem) that essentially suggests that no one method can perform ideally in all circumstances.  Therefore, when we are faced with the analysis of some new dataset there is no way to know the optimal strategy to apply ahead of time. Sometimes simple analysis methods can work optimally (as well as run quickly with interpretable results), however other times much more complicated methods are required that may have different trade-offs. 

<img src="images/ml_algorithms.png" />

### The Decision Tree algorithm

In this analysis/notebook we have decided to focus on [decision trees](https://en.wikipedia.org/wiki/Decision_tree). We have selected decision trees because:
- Easier to understand and an intuitive ML modeling approach
- The models produced by these systems are easy to interpret and apply
- Decision trees also provide the foundation for more advanced ensemble methods suc has Random Forest, Adaptive Boosting (AdaBoost), Gradient Boosting Machine (XGBoost)

However decision trees are often suseptible to bias, and even minor instance sampling differences in the training set can alter the trained model, impacting model generalizability and reproducibility. 

### Decision Tree Modeling

<img src="images/dt_terminology.png" />


### Decision Trees Training

- Beging at 'root' node
- Recursevly finds a variable that best divides data into outcomes. __Hunt's Algorithm__
- ‘best’ variable is determined heuristically
    - CART (Classification and Regression Trees) → uses Gini Index(Classification) as metric
- Heuristics: produce spits as homogeneous (pure) as possible in terms of outcome labels
- Stop criterion: Max depth or purity of labels in each leaf
    - helps to prevent overfitting 
    
### Decision Trees Splitting


<img src="images/dt_splitting.png" />



### Decision Tree: Fitting with Splits (examples)

- case = green
- control = blue
- 2 variables = X and Y

<img src="images/firstlevel.png" />
<img src="images/secondlevel.png" />
<img src="images/thirdlevel.png" />
<img src="images/fourthlevel.png" />
<img src="images/fifthlevel.png" />


The scikit-learn package implements a decision tree algorithm that is an

In [ ]:
# Train a decision tree model. (no hyperparameters set other than random seed, i.e. default hyperparameters applied)
dt = tree.DecisionTreeClassifier(random_state=randSeed)
dt = dt.fit(x_train, y_train)

# Determine model's predictions on training data
train_pred = dt.predict(x_train)
print("Training Accuracy:",metrics.accuracy_score(y_train, train_pred))

# Determine model's predictions on testing data
test_pred = dt.predict(x_test)
print("Testing Accuracy:",metrics.accuracy_score(y_test, test_pred))

* Using a basic accuracy metric (i.e. (TP+TN)/(TP+TN+FP+FN)) our decision tree yields a training accuracy and a testing accuracy.  Keep in mind that our dataset is imbalanced so this evaluation metric is not optimal and our interpretation may be biased. We will revist this below. 

## Hyperparameter Sweep
Most any ML method has 'hyper' or 'run' parameters that the user can tune/adjust, modifying how the algorithm functions. The process of tweaking and optimizing these parameters to improve ML performance is known as [hyperparameter optimization](https://en.wikipedia.org/wiki/Hyperparameter_optimization). A hyperparameter sweep mostly commonly includes idenifying which algorithm parameters are known to, or are likely to, impact performance on a given dataset, and then varying the setting of these one or more paramters in separate runs of the algorithm.  This step is not only important for squeezing the maximum performance out of ML methods, but also in fairly compairing the performance of different ML methods on given datasets.  

Out of necessity, almost all ML algorithms have default hyperparameters specified (related to [default arguments](https://en.wikipedia.org/wiki/Default_argument). First and foremost, default parameters are specified so that the algorithm is able to run even if the user forgets to specify these hyperparamters. They are often set with the intention of testing out the method on a 'toy' or demonstration dataset included with the software, or they are set as simple, accessible placeholders to ensure that the first time a user runs the software it runs smoothly and quickly. However default parameters are certainly not guarenteed (if even likely) to lead to optimal ML performance on a given dataset. 

The most common and basic hyperparameter sweep strategies include a [grid search](https://medium.com/datadriveninvestor/an-introduction-to-grid-search-ff57adcc0998) or a [random search](https://en.wikipedia.org/wiki/Random_search), but other more sophisticated searches are also employed. 

Like with many other things in ML, there is no absolute right way to perform a hyperparameter sweep. One could easily waste a large amount of computational time on an exhaustive serach of hyperparameter settings and combinations without improving performance much, or at all. However some degree of hyperparameter searching outside of using the default hyperparameter settings is considered essential to ML best practices. 

### Decision Tree - Random Sweep of Major Hyperparameters

<img src="images/hyper.png" />

We employ a simple randomized hyperparameter search (with built in, internal cross validation - based on our training sets)

In [ ]:
# Prepare a range/set of hyperparameter values for each
param_grid = {"max_depth": [3, 4, 5, 6, 7, 9, 10, None], "min_samples_split": randint(2, 100), "min_samples_leaf": randint(1, 100), "criterion": ["gini", "entropy"]}

model = tree.DecisionTreeClassifier(random_state=randSeed)
#Specifics of the random sweep - up to 100 randomly selected hyperparameter combinations
hp_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=100, random_state=randSeed)
hp_search.fit(x_train, y_train)

# summarize the results of the random parameter search
print("**************************")
print(hp_search.best_score_)
print("************************************************************************************")
print("The model with the best accuracy was built with the following hyperparameter values.")
print(hp_search.best_params_)

* We retrain a decision tree on our training data using these 'optimal' parameters. 

In [ ]:
# Train a new decision tree model. (hyperparameters set to those that yielded the best accuracy in the random sweep)
dt = tree.DecisionTreeClassifier(max_depth=hp_search.best_params_['max_depth'], 
                                 min_samples_leaf=hp_search.best_params_['min_samples_leaf'], 
                                 min_samples_split=hp_search.best_params_['min_samples_split'], 
                                 criterion=hp_search.best_params_['criterion'],random_state=randSeed)
dt = dt.fit(x_train, y_train)

# Determine model's predictions on training data
dt_train_pred = dt.predict(x_train)
print("Training Accuracy:",metrics.accuracy_score(y_train, dt_train_pred))

# Determine model's predictions on testing data
dt_test_pred = dt.predict(x_test)
print("Testing Accuracy:",metrics.accuracy_score(y_test, dt_test_pred))

## Model Evaluation
Now that we have trained models, how do we best evaluate and compare the performance of these models?  Proper evaluation is critical to making good conclusions. Many metrics and measures of model 'goodness' exist. Part of what differentiates these evaluation methods is understanding the assumptions being made by a given metric, or what the metric prioritizes as important for model goodness. A nice review of key evaluation metrics can be found [here](http://www.davidsbatista.net/blog/2018/08/19/NLP_Metrics/). 

Below we will apply some different evaluation methods to the decision tree model that was trained using the 'optimal' parameters settings indentified in the random sweeep (we will leave out the random forest evaluation here for brevity). All evaluations will focus on predictive performance on the testing data. 

The metrics are typically based on calculations using the different possible classification predictions that can be made: True Positives (TP), True Negative (TN), False Positive (FP), and False Negative (FN).  See [here](https://towardsdatascience.com/the-mystery-of-true-positive-true-negative-false-positive-and-false-negative-fd73c78c905a) to review TP, TN, FP, FN.

Let's start by calculating each of these for our test data using our 'optimized' decision tree model. Recall that our dataset is imbalanced and there are more 'negatives' than 'positives'. 

In [ ]:
#Plotting confusion matrix
def plot_confusion_matrix(y_test, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    # Only use the labels that appear in the data
    # For unique lebel read this:
    # https://scikit-learn.org/stable/modules/generated/sklearn.utils.multiclass.unique_labels.html#sklearn.utils.multiclass.unique_labels
    
    classes = unique_labels(y_test, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
#Calculate confusion matrix (true outcomes, predicted outcomes)
TN, FP, FN, TP = confusion_matrix(y_test, dt_test_pred).ravel()

print("TP = {0}".format(TP))
print("FP = {0}".format(FP))
print("FN = {0}".format(FN))
print("TN = {0}".format(TN))

plot_confusion_matrix(y_test, dt_test_pred, classes=np.unique(y_train), title='Confusion matrix')

These values (TP,FP,FN,TN) are components used to calculate a number of different evaluation metrics for classification tasks:

* Accuracy = (TP+TN)/(TP+TN+FP+FN)
* Precision (a.k.a. Sensitivity)= TP/(TP+FP)
* Recall (a.k.a. True Positive Rate) = TP/(TP+FN)
* Specificity = TN/(TN+FP)
* False Positive Rate = FP/(FP+TN)

* F1 Score = 2*(Precision * Recall)/(Precision + Recall)
* Balanced Accuracy = (Sensitivity + Specificity)/2

We will calculate these for our 'optimized' decision tree model.  Be aware that a different set of evaluation metrics would be used for continuous-valued outcomes (i.e. regression tasks). 

In [ ]:
# Determine model's predictions on testing data
print("All metrics report performance on testing data!")
print("***********************************************")
print("Accuracy:",metrics.accuracy_score(y_test, dt_test_pred))
print("***********************************************")

#Generate a classification report
report = classification_report(y_test, dt_test_pred)
print (report)
print("***********************************************")
specificity = TN/(TN+FP)
sensitivity = TP/(TP+FP)
print("Specificity: "+str(specificity))
print("***********************************************")
print("False Positive Rate: "+str(FP/(TN+FP))+ "   Note: Lower is better.")
print("***********************************************")
print("Balanced Accuracy: "+str((sensitivity+specificity)/2))

* For imbalanced datasets, balanced accuracy is less biased in comparing performance as it weights the accurate prediction of both classes equally. 

### ROC Curve and AUC metric
The ROC curve is a metric used for the evaluation of binary classification models, where we evaluate how the model would perform if different cuttoff thresholds (of predicted class probability) between the two classes were considered. This is useful when we don't necessarily know which class is more important to predict accurately. For example, if we wanted to evaluate how well a test for a highly infectious disease worked, we would care much more about identifying positive individuals correctly than negative individuals, even at the expense of additional false positives.

To calculate an ROC curve first we need to get the class prediction probabilites from the model, rather than just the predicted classes, since these predicted classes were decided based on a predetermined classification threshold. 

In [ ]:
#Determine probabilities of class predictions for each test instance (this will be used much later in calculating an ROC curve)
probas_ = dt.fit(x_train, y_train).predict_proba(x_test)

# Compute ROC curve and area the curve
fpr, tpr, thresholds = metrics.roc_curve(y_test, probas_[:, 1])
roc_auc = auc(fpr, tpr)

#Plot the ROC Curve and include AUC in figure. 
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

### Precision Recall Plot - An alternative to ROC when there is class imbalance
While we have successuflly generated an ROC curve above, when there is a moderate to large class imbalance it is considered better practice to evaluate models with a precision/recall plot (PRC) ([see here](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4349800/)).

In [ ]:
dot_data = tree.export_graphviz(dt, out_file=None,feature_names=hcc_df.drop('target', axis=1).columns, class_names=['0','1']) 
graph = graphviz.Source(dot_data) 
graph.render("decisionTree") 
graph